# Search Algorithms

In this project, the aim is to solve a problem using various search algorithms and compare their qualities. BFS, IDS and A* are the algorithms used for this problem.

## Problem Description

A player starts at location (0,0) of a $n\times m$ map. some cells are blocked by obstacles. at each steps, the player can choose to move up, down, left or right if its path is not blocked by walls or obstacles. each cell may contain one of the following elements:

  - <b>Potion:</b> When player enters a cell with potion, he collects the potion.
    
    
  - <b>Pill:</b> When player enters a cell with pill, he consumes the pill and a new player appears at location (n-1,0). afterwards, only one of the players may move at each step 
    
The goal is to gather all players at location (n-1,m-1) and collect all of the potions in the map.

## Modeling

At first, the problem should be modeled as a graph so that the search algorithms can be applied to it in future steps. Each node representes one ditinct state of the map. Bellow, the components of the model is explained:

 - <b>Initial State:</b> The initial state of the map is given as input in this problem, and the player always starts at bottom left corner of the map.
 
 
 - <b>Actions:</b> At each step,one of the players can move either up, down, left or right. However, sometimes the players may be prevented from taking certain actions due to walls and obstacles.
 
 
 - <b>Transition Model:</b> A state is recognized by this properties: 1- location of each player, 2- remaining potions, 3-location of each component(potion-pill-obstacle..) our transition model takes a state and an action as input and determines the next state properties using the rules described in problem description. if an action is impossible to take, the function returns 0 as output. 
 
 
 - <b>Goal State:</b> The goal state is when all of the players have reached the top right corner of the map and collected all potions in the map.
 
 
 - <b>Path Cost:</b> In this problem, the cost of all actions are equal
 

### Model Implementation

In [1]:
import time
import copy as cp
import heapq as hp
import bisect
from collections import deque
PLAYER='P'
POTION='M'
PILL='D'
OBSTACLE='O'

- <b>States:</b> we use dictionary to store the properties of each state. we choose dictionary because of its simplicity and built-in functions

<b>note:</b> Even though in was possible to obtain the number of potions by location of each component, we choose to store it separately because is used many times and we want our functions to be optimal.

 - <b>Initiat State:</b>

In [2]:
def read_initial_state(file_name):
    file = open(file_name)
    row,col=file.readline().split()
    potion_count,pill_count=file.readline().split()
    initial_map={"row": int(row) , "col": int(col) , "potion_count": int(potion_count), "ploc":[]}
    for _ in range(int(potion_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=POTION
    for _ in range(int(pill_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=PILL
    obstacle_count=file.readline()
    for _ in range(int(obstacle_count)):
        x,y=file.readline().split()
        initial_map[(int(x),int(y))]=OBSTACLE
    initial_map["ploc"].append((0,0))
    if((0,0) in initial_map):
        if(initial_map[(0,0)]==PILL):
            initial_map["ploc"].append((initial_map["row"]-1,0))
        elif(initial_map[(0,0)]==POTION):
            current_state["potion_count"]-=1;
    return initial_map

 - <b>Actions:</b>

In [3]:
UP=(1,0)
DOWN=(-1,0)
RIGHT=(0,1)
LEFT=(0,-1)
ACTIONS=[RIGHT,UP,DOWN,LEFT]
ACT_DICT={UP:"U",RIGHT:"R",DOWN:"D",LEFT:"L"}

 - <b>Transition Model:</b>

In [4]:
def not_in_range(loc,loc_range):
    return loc[0]>=loc_range[0] or loc[1]>=loc_range[1] or loc[0]<0 or loc[1]<0
    

def move(current_state,player_number,direction):
    current_state=cp.deepcopy(current_state)
    current_location=current_state["ploc"][player_number]
    future_location=(current_location[0]+direction[0],current_location[1]+direction[1])
    if(future_location in current_state):
        if(current_state[future_location]==POTION):
            current_state["potion_count"]-=1;
        elif(current_state[future_location]==PILL):
            current_state["ploc"].append((current_state["row"]-1,0))
        elif(current_state[future_location]==OBSTACLE):
            return 0
    elif(not_in_range(future_location,(current_state["row"],current_state["col"]))):
        return 0
    current_location=current_state["ploc"][player_number]=future_location
    if(current_location in current_state):
        del current_state[current_location]
    return current_state

- <b>Goal State:</b>

In [5]:
def is_goal_state(state):
    for loc in state["ploc"]:
        if(loc!=(state["row"]-1,state["col"]-1)):
            return False
    if(state["potion_count"]==0):
        return True
    else:
        return False

# Breadth First Search (BFS)

## Algorithm
bellow the BFS algorithm is implemented. a dictionary is used to store explored states because it is faster to check the existance of a key in dictionary. Since a dictionary cannot be used as a key, we define a function "encode" that takes an state and turn it into hashable format and returns the hashed dictionary.
## Pros and Cons
pros:
 - Always returns the optimal solution
 
cons:
 - Requires more space than IDS (of order O($b^m$))
 - Visits more states and is slower than A*(depends on heuristic)

## Implementation

In [6]:
def encode(state):
    encoded=state.copy()
    encoded.pop("ploc",None)
    encoded.pop("order",None)
    encoded.pop("level",None)
    for player in range(len(state["ploc"])):
        encoded[player]=state["ploc"][player]
    return hash(tuple(encoded.items()))

def BFS(file_address):
    total_visited_states=1;
    frontier_queue=deque()
    visited_states=set()
    frontier_queue.append(read_initial_state(file_address))
    frontier_queue[0]["order"]=""
    while(frontier_queue):
        current_state=frontier_queue.popleft()
        if((encode(current_state) in visited_states)):
            continue
        visited_states.add(encode(current_state))
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                total_visited_states+=1
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                if(is_goal_state(new_state)):
                    return (new_state["order"],total_visited_states)
                frontier_queue.append(new_state) 
    return (0,0)

## Tests

In [7]:
tic=time.time()
BFS("Tests/test1.in")
toc=time.time()
time1=toc-tic
tic=time.time()
BFS("Tests/test1.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =BFS("Tests/test1.in")
toc=time.time()
time3=toc-tic
average_time_BFS_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\nPath len = ",len(result_test1.split()) ,"\naverage time= ",average_time_BFS_test1,"\nvisited_states=",visited_states1)

Path=  0R 0R 0R 0U 0U 0U 1D 1R 1R 1R 1U 
Path len =  11 
average time=  0.09492031733194987 
visited_states= 3535


In [8]:
tic=time.time()
BFS("Tests/test2.in")
toc=time.time()
time1=toc-tic
tic=time.time()
BFS("Tests/test2.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test2,visited_states2=BFS("Tests/test2.in")
toc=time.time()
time3=toc-tic
print(time3)
average_time_BFS_test2=(time1+time2+time3)/3
print("Path=" ,result_test2, "\nPath len = ",len(result_test2.split()) ,"\naverage time= ",average_time_BFS_test2,"\nvisited_states=",visited_states2)

26.667701721191406
Path=  0R 0R 0U 0U 0R 0U 1D 1R 1R 1R 1U 
Path len =  11 
average time=  25.555316130320232 
visited_states= 634400


In [9]:
tic=time.time()
BFS("Tests/test3.in")
toc=time.time()
time1=toc-tic
print(time1)
tic=time.time()
BFS("Tests/test3.in")
toc=time.time()
time2=toc-tic
print(time2)
tic=time.time()
result_test3,visited_states3=BFS("Tests/test3.in")
toc=time.time()
time3=toc-tic
print(time3)
average_time_BFS_test3=(time1+time2+time3)/3
print("Path=" ,result_test3, "\nPath len = ",len(result_test3.split()) ,"\naverage time= ",average_time_BFS_test3,"\nvisited_states=",visited_states3)

24.25251817703247
23.89982008934021
23.68352246284485
Path=  0U 0U 0R 0R 0D 0R 0R 0R 0U 0U 0U 0U 1R 1R 1R 1D 1R 1R 1U 
Path len =  19 
average time=  23.945286909739178 
visited_states= 553736


## Overall Results
|property|test1|test2|test3|
|:-:|:---:|:---:|:---:|
|**Time**          |95ms|25.5s|24s|
|**States Visited**|3535|634400|553736|
|**Path Depth** |11|11|19|

# Iterative Deepening Search (IDS)

## Algorithm
bellow the IDS algorithm is implemented. "encode" function from BFS is used here as well. the IDS functions uses DFS as a subroutine, it runs DFA but specifies the max depth DFS should explore and starting from 1, increases the depth after each run untill goal state is reached  
## Pros and Cons
pros:
 - Always returns the optimal solution
 - Uses less space than A* and BFS
 
cons:
 - Visits more states and is slower than A*(depends on heuristic)
 - It is slower than BFS because it visits some states several times, however, it has the same time complexity. 

## Implementation

In [10]:
def DFS(initial_state,max_level):
    total_visited_states=1;
    frontier_queue=[]
    visited_states={}
    frontier_queue.append([0,initial_state])
    initial_state["order"]=""
    while(frontier_queue):
        added_level,current_state=frontier_queue.pop()
        if((encode(current_state) in visited_states)):
            if(visited_states[encode(current_state)]<=added_level):
                continue
        visited_states[encode(current_state)]=added_level
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                if(is_goal_state(new_state)):
                    return (new_state["order"],total_visited_states)
                total_visited_states+=1;
                if(added_level+1<max_level):
                    frontier_queue.append([added_level+1,new_state]) 
    return (0,total_visited_states)
def IDS(file_address):
    total_states=0
    level=1
    start_state=read_initial_state(file_address)
    start_state["order"]=""
    res,total=DFS(start_state,level)
    total_states+=total
    while res==0:
        level+=1
        res,total=DFS(start_state,level)
        total_states+=total
    return (res,total_states)

## Tests

In [11]:
tic=time.time()
IDS("Tests/test1.in")
toc=time.time()
time1=toc-tic
tic=time.time()
IDS("Tests/test1.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =IDS("Tests/test1.in")
toc=time.time()
time3=toc-tic
average_time_IDS_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\nPath len = ",len(result_test1.split()) ,"\naverage time= ",average_time_IDS_test1,"\nvisited_states=",visited_states1)

Path=  0R 1D 1U 1R 1R 1R 0R 0R 0U 0U 0U 
Path len =  11 
average time=  0.5442923704783121 
visited_states= 21609


In [12]:
tic=time.time()
IDS("Tests/test2.in")
toc=time.time()
time1=toc-tic
tic=time.time()
IDS("Tests/test2.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test2,visited_states2 =IDS("Tests/test2.in")
toc=time.time()
time3=toc-tic
average_time_IDS_test2=(time1+time2+time3)/3
print("Path=" ,result_test2, "\nPath len = ",len(result_test2.split()) ,"\naverage time= ",average_time_IDS_test2,"\nvisited_states=",visited_states2)

Path=  1D 1R 1R 1U 1R 0R 0R 0U 0U 0U 0R 
Path len =  11 
average time=  68.0701032479604 
visited_states= 1768998


In [13]:
tic=time.time()
IDS("Tests/test3.in")
toc=time.time()
time1=toc-tic
tic=time.time()
IDS("Tests/test3.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test3,visited_states3=IDS("Tests/test3.in")
toc=time.time()
time3=toc-tic
print(time3)
average_time_IDS_test3=(time1+time2+time3)/3
print("Path=" ,result_test3, "\nPath len = ",len(result_test3.split()) ,"\naverage time= ",average_time_IDS_test3,"\nvisited_states=",visited_states3)

112.91247177124023
Path=  0U 0U 0R 0R 1R 1R 1R 1D 1U 1R 1R 0D 0R 0R 0R 0U 0U 0U 0U 
Path len =  19 
average time=  112.48963658014934 
visited_states= 2534691


## Overall Results
|property|test1|test2|test3|
|:-:|:---:|:---:|:---:|
|**Time**          |544ms|68s|112.5s|
|**States Visited**|21609|1768998|2534691|
|**Path Depth** |11|11|19|

# A*

## Algorithm
bellow the A* algorithm is implemented. the method used to keep the track of visited states is similar to previous parts. For this algorithm, a heap is used to extract the state with minimum cost at each step. the cost of a state is defined as follows:

$$f(n)=g(n)+h(n)$$

where $h(n)$ is the heuristic function that estimates the cost to reach the goal and $g(n)$ is the cost of reaching n from start state.

### heuristic function
We need to define a heuristic that gives us a good estimation of the cost of reaching the goal and is also consistent. To find a good heuristic, we must first pay attention to the two properties of a goal state:
 - **All players should be at location (n-1,m-1)**
 - **All potions should be collected**
 
if we pay attention to just the first part, we know that $location(final)=(row-1,col-1)$, so we can define the following heuristic:

$$h_{distance}(n)=\sum_{player}d_{Manhattan}(locatin(player),location(final))$$

However, this is not a good heuristic because it favours closing the distance to final cell at the cost of leaving the potions behind. so we need to modify it to include the penalty of leaving potions behind.
if we show a location as (x,y), we define the following symbols:
 - **$player_{min x}$**: player with smallest x
 - **$player_{min y}$**: player with smallest y
 - **$potion_{min x}$**: potion with smallest x
 - **$potion_{min y}$**: potion with smallest y
 - $Positive(x)$: returns x if x is positive, otherwise returns 0
 
then we define the following function(it is not a heuristic):

$$h_{potion}(n)=Positive(player_{min x}-potion_{min x})+Positive(player_{min y}-potion_{min y}) $$

our final heuristic is:

$$h(n)=h_{potion}(n)+h_{distance}(n) $$

Now we prove that this heuristic is consistent:
take A=current state and C=future state, we know that cost(A to C) is 1.
we prove consistency for action sets {Down,Left} and {Up,Right} seprately. notice that since both $h_{distance}(n)$ is distance based and $h_{potion}(n) $ is the sum of one horizontal and one vertical distance and they cannot change simultaneously after one step, if no pill is taken, they increase or decrase by 1 after each action.
 - **UP or Right:**
  - **Pill is taken:**
      $$h_{potion}(A)-h_{potion}(C)\leq Positive(player(A)_{min y}-potion_{min y})\leq n-1$$
      $$h_{distance}(A)- h_{distance}(C)=-(n-2)$$
      $$\rightarrow h(A)-h(C)\leq 1$$
  - **Pill is not taken:**
      $$h_{potion}(A)-h_{potion}(C)\leq 0$$
      $$h_{distance}(A)- h_{distance}(C)=1$$
      $$\rightarrow h(A)-h(C)\leq 1$$
 - **Down or Left:**
  - **Pill is taken:**
      $$h_{potion}(A)-h_{potion}(C)\leq (player(A)_{min y}-potion_{min y})\leq n-1$$
      $$h_{distance}(A)- h_{distance}(C)=-(n)$$
      $$\rightarrow h(A)-h(C)\leq -1$$
  - **Pill is not taken:**
      $$h_{potion}(A)-h_{potion}(C)\leq 1$$
      $$h_{distance}(A)- h_{distance}(C)=-1$$
      $$\rightarrow h(A)-h(C)\leq 0$$
      
so in all cases, $h(A)-h(C)\leq cost(A to C)$ and our heuristic is consistent

## Pros and Cons
pros:
 - Always returns the optimal solution with a consistent heuristic
 - Can be very faster and visit less states than BFS and IDS with a good heuristic function
 
cons:
 - It may be difficult and complicated to find a good heuristic for a problem.
 - Requires more space than IDS.

## Implementation

In [27]:
def heuristic(state):
    player_steps=0
    min_row=state["row"]-1
    min_col=state["col"]-1
    col=min_col
    row=min_row
    max_dist_row=0
    max_dist_col=0
    for loc in state["ploc"]:
        player_steps=row-loc[0]+col-loc[1]
        if row<loc[0]:min_row=loc[0]
        if col<loc[1]:min_col=loc[1]
    for key,val in state.items():
        if (type(key)==tuple):
            if(val==POTION):
                row=min_row-key[0]
                col=min_col-key[1]
                if col>max_dist_col: max_dist_col=col
                if row>max_dist_row: max_dist_row=row
                
    return player_steps+(max_dist_row+max_dist_col)

In [15]:
def A_Star(file_address,alpha=1):
    seen_states=0;
    frontier_queue=[]
    visited_states={}
    initial_state=read_initial_state(file_address)
    hp.heappush(frontier_queue,(alpha*heuristic(initial_state),0,initial_state))
    visited_states[encode(initial_state)]=True
    initial_state["order"]=""
    initial_state["level"]=0
    while(frontier_queue):
        cost,order,current_state=hp.heappop(frontier_queue)
        visited_states[encode(current_state)]=True
        if(is_goal_state(current_state)):
            return (current_state["order"],seen_states)
        for player in range(len(current_state["ploc"])):
            for action in ACTIONS:
                seen_states+=1
                new_state=move(current_state,player,action)
                if(new_state==0):
                    continue
                if((encode(new_state) in visited_states)):
                    continue
                new_state["order"]=current_state["order"]+" "+str(player)+ACT_DICT[action]
                new_state["level"]=current_state["level"]+1
                bisect.insort(frontier_queue,(alpha*heuristic(new_state)+new_state["level"],seen_states,new_state)) 
    return 0

## Tests

In [28]:
tic=time.time()
A_Star("Tests/test1.in")
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test1.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =A_Star("Tests/test1.in")
toc=time.time()
time3=toc-tic
average_time_A_Star_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\nPath len = ",len(result_test1.split()) ,"\naverage time= ",average_time_A_Star_test1,"\nvisited_states=",visited_states1)

Path=  0R 0R 1D 1R 1R 1R 0R 0U 1U 0U 0U 
Path len =  11 
average time=  0.05764174461364746 
visited_states= 3292


In [29]:
tic=time.time()
A_Star("Tests/test2.in")
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test2.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test2,visited_states2 =A_Star("Tests/test2.in")
toc=time.time()
time3=toc-tic
average_time_A_Star_test2=(time1+time2+time3)/3
print("Path=" ,result_test2,"\nPath len = ",len(result_test2.split()) , "\naverage time= ",average_time_A_Star_test2,"\nvisited_states=",visited_states2)

Path=  1D 1R 1R 1U 0R 0R 0U 0U 0U 1R 0R 
Path len =  11 
average time=  2.339558998743693 
visited_states= 85440


In [30]:
tic=time.time()
A_Star("Tests/test3.in")
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test3.in")
toc=time.time()
time2=toc-tic
tic=time.time()
result_test3,visited_states3=A_Star("Tests/test3.in")
toc=time.time()
time3=toc-tic
average_time_A_Star_test3=(time1+time2+time3)/3
print("Path=" ,result_test3, "\nPath len = ",len(result_test3.split()) ,"\naverage time= ",average_time_A_Star_test3,"\nvisited_states=",visited_states3)

Path=  0U 0U 0R 0R 1R 0D 1R 0R 1R 0R 0R 0U 1D 0U 1U 1R 0U 0U 1R 
Path len =  19 
average time=  0.3070720036824544 
visited_states= 9940


## Overall Results
|property|test1|test2|test3|
|:-:|:---:|:---:|:---:|
|**Time**          |58ms|2.33s|307ms|
|**States Visited**|3292|85440|9940|
|**Path Depth** |11|11|19|

# Weighted A*

We may want to loose optimality to gain more speed. Weighted A* algorithm can be used to do that. It is faster than A* but will not always return the optimal solution. it uses $\alpha\times h(n)$ ($\alpha>1$) instead of $h(n)$ as heuristic, so we may loose consistancy and optimal solution with some $\alpha$

## $\alpha$ = 1.3

In [31]:
alpha=1.3

In [32]:
tic=time.time()
A_Star("Tests/test1.in",alpha)
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test1.in",alpha)
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =A_Star("Tests/test1.in",alpha)
toc=time.time()
time3=toc-tic
average_time_A_Star_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\nPath len = ",len(result_test1.split()) ,"\naverage time= ",average_time_A_Star_test1,"\nvisited_states=",visited_states1)

Path=  0R 0R 1D 1R 1R 1U 0R 0U 0U 1R 0U 
Path len =  11 
average time=  0.020209868748982746 
visited_states= 852


In [33]:
tic=time.time()
A_Star("Tests/test2.in",alpha)
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test2.in",alpha)
toc=time.time()
time2=toc-tic
tic=time.time()
result_test2,visited_states2 =A_Star("Tests/test2.in",alpha)
toc=time.time()
time3=toc-tic
print(time3)
average_time_A_Star_test2=(time1+time2+time3)/3
print("Path=" ,result_test2,"\nPath len = ",len(result_test2.split()) , "\naverage time= ",average_time_A_Star_test2,"\nvisited_states=",visited_states2)

0.2846188545227051
Path=  1D 1R 0R 0R 0U 0U 0U 1R 1R 0R 1U 
Path len =  11 
average time=  0.29398004213968915 
visited_states= 11668


In [34]:
tic=time.time()
A_Star("Tests/test3.in",alpha)
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test3.in",alpha)
toc=time.time()
time2=toc-tic
tic=time.time()
result_test3,visited_states3=A_Star("Tests/test3.in",alpha)
toc=time.time()
time3=toc-tic
average_time_A_Star_test3=(time1+time2+time3)/3
print("Path=" ,result_test3, "\nPath len = ",len(result_test3.split()) ,"\naverage time= ",average_time_A_Star_test3,"\nvisited_states=",visited_states3)

Path=  0U 0U 0R 0R 1R 0D 1R 1R 1D 1R 1R 1U 0R 0R 0R 0U 0U 0U 0U 
Path len =  19 
average time=  0.015844027201334637 
visited_states= 332


## Overall Results
||test1|test2|test3|
|:-:|:---:|:---:|:---:|
|**Time**          |20ms|300ms|16ms|
|**States Visited**|852|11668|332|
|**Path Depth** |11|11|19|

## $\alpha$ = 5

In [35]:
alpha=5

In [36]:
tic=time.time()
A_Star("Tests/test1.in",alpha)
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test1.in",alpha)
toc=time.time()
time2=toc-tic
tic=time.time()
result_test1,visited_states1 =A_Star("Tests/test1.in",alpha)
toc=time.time()
time3=toc-tic
average_time_A_Star_test1=(time1+time2+time3)/3
print("Path=" ,result_test1, "\nPath len = ",len(result_test1.split()) ,"\naverage time= ",average_time_A_Star_test1,"\nvisited_states=",visited_states1)

Path=  0R 0R 1D 1R 1R 1R 1U 0R 0U 0U 0U 
Path len =  11 
average time=  0.0061511993408203125 
visited_states= 132


In [37]:
tic=time.time()
A_Star("Tests/test2.in",alpha)
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test2.in",alpha)
toc=time.time()
time2=toc-tic
tic=time.time()
result_test2,visited_states2 =A_Star("Tests/test2.in",alpha)
toc=time.time()
time3=toc-tic
print(time3)
average_time_A_Star_test2=(time1+time2+time3)/3
print("Path=" ,result_test2,"\nPath len = ",len(result_test2.split()) , "\naverage time= ",average_time_A_Star_test2,"\nvisited_states=",visited_states2)

0.034203290939331055
Path=  0U 0U 0R 0R 0D 1R 2R 2R 2R 1R 1R 0U 0U 0R 
Path len =  14 
average time=  0.044819911321004234 
visited_states= 1404


In [38]:
tic=time.time()
A_Star("Tests/test3.in",alpha)
toc=time.time()
time1=toc-tic
tic=time.time()
A_Star("Tests/test3.in",alpha)
toc=time.time()
time2=toc-tic
tic=time.time()
result_test3,visited_states3=A_Star("Tests/test3.in",alpha)
toc=time.time()
time3=toc-tic
average_time_A_Star_test3=(time1+time2+time3)/3
print("Path =" ,result_test3,"\nPath len = ",len(result_test3.split()) , "\naverage time = ",average_time_A_Star_test3,"\nvisited_states = ",visited_states3)

Path =  0U 0U 0R 0R 1R 0D 1R 1R 1R 1R 0R 0R 0R 0U 0U 0U 0L 0L 0U 0R 0R 
Path len =  21 
average time =  0.013657728830973307 
visited_states =  244


## Overall Results
||test1|test2|test3|
|:-:|:---:|:---:|:---:|
|**Time**          |6ms|45ms|14ms|
|**States Visited**|132|1404|244|
|**Path Depth** |11|14|21|

# Conclusion

In general, the algorithm we choose to use depends highly on our situation, BFS is faster than IDS but needs more space. IDS, while it is slower than BFS, uses less space so it is suitable when we have a very limited memory. A star can be faster than BFS and IDS if a good heuristic is choosen, however, finding a good heuristic can be challenging and A star also requires more space than IDS. With Weighted A star , we loose the optimality but get the result way faster than normal A star.

Here are the overall results for each test:

## - Test1



|**Method**|**Avg Time**|**Path Depth**|**States Visited**|
|:-:|:---:|:---:|:---:|
|**BFS**          |95ms|11|3535|
|**IDS**|544ms|11|21609|
|**A_star** |58ms|11|3292|
|**Weighted A_star ($\alpha = 1.3$)** |20ms|11|852|
|**Weighted A_star ($\alpha = 5$)** |6ms|11|132|

## - Test2



|**Method**|**Avg Time**|**Path Depth**|**States Visited**|
|:-:|:---:|:---:|:---:|
|**BFS**          |25.5s|11|634400|
|**IDS**|68s|11|1768998|
|**A_star** |2.33s|11|85440|
|**Weighted A_star ($\alpha = 1.3$)** |300ms|11|11668|
|**Weighted A_star ($\alpha = 5$)** |45ms|14|1404|

## - Test3



|**Method**|**Avg Time**|**Path Depth**|**States Visited**|
|:-:|:---:|:---:|:---:|
|**BFS**          |24s|19|553736|
|**IDS**|112.5s|19|2534691|
|**A_star** |307ms|19|9940|
|**Weighted A_star ($\alpha = 1.3$)** |16ms|19|332|
|**Weighted A_star ($\alpha = 5$)** |14ms|21|244|